In [ ]:
## (Labeling - I) ##

# This part is done manually based on the "annotation_pipeline/mmr_selected/Q152_mmr_selected.csv" dataset
# A new match column was created and binary hand labeling was performed based on the combined scores (question ID and likert scale)
# Later on the these initial labels will be enriched after the first training run
# The resulting labeled dataset is saved as "Q154_mmr_selected_labeled.csv" in the current directory
# The dataset was the processed to exclude the accompanying metadata and only include the corresponding hashes, and saved as "Q154_mmr_selected_labeled_combined.csv"

%run clean_labeled_data.py

Loading Q154_mmr_selected_labeled.csv ...
Loading ../top_scored_sentences.csv ...
Creating combined sentence column...
Saving to Q154_mmr_selected_labeled_combined.csv ...
Done.


In [1]:
## (Training) ##

# Note: This code was used twice:
# (1) The first run was made right after the hand labeling was done.
# (2) The second training used a combined labeled dataset of hand labeled data and the hand-picked accurate predictions from the first run by relabeling. 

# Load and filter labeled data, focusing on relevant combined_labels and positive matches.
# Perform stratified train-validation split by label with a fixed holdout fraction.
# Augment training data by adding synonym-replaced sentences to improve robustness.
# Encode sentences with SBERT embeddings and train a CatBoost multi-class classifier.
# Evaluate the model on validation data and save the trained model and label mappings.

%run train_catboost_labeled.py

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\secki\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\secki\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Loading data...
Loading SBERT model...
Encoding all sentences for embedding hash computation...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

Splitting data stratified by embedding_hash...
Training samples before augmentation: 91
Validation samples before filtering: 61
Performing synonym augmentation on training set...
Training samples after augmentation: 364
Encoding training sentences after augmentation...


Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Encoding validation sentences...


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Encoding original training sentences for permutation tests...


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Filtering validation samples too close to training samples...
Validation samples after cosine similarity filtering: 60
Training CatBoost classifier...
0:	learn: 1.3680838	test: 1.3797562	best: 1.3797562 (0)	total: 249ms	remaining: 8m 16s
100:	learn: 0.3925077	test: 0.8876986	best: 0.8876986 (100)	total: 6.25s	remaining: 1m 57s
200:	learn: 0.2002336	test: 0.7677308	best: 0.7677308 (200)	total: 12.4s	remaining: 1m 51s
300:	learn: 0.1259645	test: 0.6932382	best: 0.6932382 (300)	total: 18.9s	remaining: 1m 46s
400:	learn: 0.0872651	test: 0.6334601	best: 0.6334601 (400)	total: 24.5s	remaining: 1m 37s
500:	learn: 0.0667387	test: 0.5924106	best: 0.5924106 (500)	total: 29.4s	remaining: 1m 27s
600:	learn: 0.0537788	test: 0.5622478	best: 0.5622478 (600)	total: 34s	remaining: 1m 19s
700:	learn: 0.0442480	test: 0.5365683	best: 0.5365683 (700)	total: 38.6s	remaining: 1m 11s
800:	learn: 0.0376560	test: 0.5164354	best: 0.5164354 (800)	total: 43.3s	remaining: 1m 4s
900:	learn: 0.0334367	test: 0.5092457

In [3]:
# (Prediction Fine-Tuning)
# Optional keyword extraction fine-tuning code using KeyBERT
%run tune_keywords_keybert.py

Top keyphrases for Q154_1:
 - stability public order remain unshaken corner nation (score: 0.731)
 - maintaining order nation utmost importance (score: 0.667)
 - internal cohesion abstract virtue prerequisite enduring peace (score: 0.584)

Top keyphrases for Q154_2:
 - voice global decision making including fairer multilateral (score: 0.659)
 - govern internationally support reforms enhance collective voice (score: 0.657)
 - importance expanding civic channels ensure citizen feels (score: 0.593)

Top keyphrases for Q154_3:
 - countries suffocated price shocks control rise dignity (score: 0.746)
 - coordination ensure developing countries suffocated price (score: 0.728)
 - fighting rising prices utmost importance (score: 0.627)

Top keyphrases for Q154_4:
 - protecting freedom speech utmost importance society censors (score: 0.789)
 - dissent disloyalty open discourse strength threat international (score: 0.708)
 - international law continue safeguard journalists writers thinkers (score

In [3]:
## (Prediction - I) ##

# This prediction only uses the top_scored_sentences.csv dataset, which is the output of the data_prep_scoring.py script.
# The prediction dataset contains the most uncertain sentences in the previous measurements
# As stated, the accurate outputs will be used to enrich the training data for the second training run.

# Load unseen sentences excluding those already labeled in training by embedding hash.
# Use a pre-trained CatBoost model and SBERT embeddings to predict combined_labels on unseen data.
# Filter predicted sentences by confidence threshold (perc_above_chance >= threshold).
# Merge additional metadata from the full scored sentences dataset based on predicted labels.
# Display a preview and save filtered, labeled predictions to CSV for further use.

%run predict_catboost_top_score.py

Selected 4425 unseen sentences for prediction.
Encoding unseen sentences...


Encoding batches: 100%|██████████| 139/139 [00:04<00:00, 33.52it/s]


Extracting keyphrases per label for semantic filtering...
Applying semantic similarity filtering per predicted label...
Applying joint scoring and filtering...
Kept 1113 predictions after joint scoring filtering.
Saved the results to predictions/q154_predictions_top_score_filtered.csv


In [ ]:
## (Labeling - II) ##

# This function appends accurately predicted sentences from the predictions file
# to the labeled dataset based on a provided list of selected sentence hashes.
# It filters these hashes to those present in the predictions, then matches them
# with the top scored sentences and marks them as matched.
# Finally, it concatenates these new matched sentences with the existing labeled data and saves it.

# After this code is run, the model was trained the second time with the appended labeled dataset.

%run label_append_after_pred.py

Total hashes in predictions: 2869
Valid selected hashes found in predictions: 16
Appended 16 rows to labeled data and saved to 'Q154_mmr_selected_labeled.csv'.


In [ ]:
## (Training - II) ##

# After the Labeling - II code is run, the model was trained the second time with the appended labeled dataset.

In [4]:
## (Prediction - II) ##

# This code loads UNGA speech sentences and WVC metadata, excluding training data to prevent leakage.
# It encodes filtered sentences with a SentenceTransformer model, then predicts classes and confidences using a CatBoost classifier.
# Predictions with confidence above chance threshold are kept, and relevant columns are selected.
# It merges the predictions with metadata from the WVC dataset based on predicted labels.
# Finally, it previews and saves the combined prediction results to a CSV file for further use.

%run predict_catboost_unga_wvs7.py

Loading combined labeled CSV...
Extracting top keyphrases per label...
Loading SBERT model...
Loading unseen data...
Filtered unseen sentences count: 25551
Loading CatBoost model...
Encoding unseen sentences...


Encoding batches: 100%|██████████| 799/799 [00:47<00:00, 16.73it/s]


Predicting classes and probabilities...
Applying semantic similarity filtering per predicted label...
Applying joint scoring and filtering...
Kept 2745 predictions after joint scoring filtering.
Saved filtered predictions to predictions/q154_predictions_filtered.csv


In [1]:
# Predict on the UNGS address dataset
%run predict_ungs_address.py

Loading combined labeled CSV...
Extracting top keyphrases per label...
Loading SBERT model...
Loading unseen data...
Loading CatBoost model...
Encoding unseen sentences...


Encoding batches: 100%|██████████| 21/21 [00:02<00:00,  8.61it/s]


Predicting classes and probabilities...
Applying semantic similarity filtering per predicted label...
Applying joint scoring and filtering...
Kept 191 predictions after joint scoring filtering.
Saved filtered predictions to predictions/q154_predictions_ungs.csv


In [2]:
# Get the most (Q154) and second most frequent (Q155) labels for each year
%run get_q154_q155_frequencies_ungs.py

,doc_id,A_YEAR,most_frequent_label,most_frequent_count,second_most_frequent_label,second_most_frequent_count,has_second_label
0,ungs_2017,2017,Q154_2,10,Q154_4,10,True
1,ungs_2018,2018,Q154_2,15,Q154_1,10,True
2,ungs_2019,2019,Q154_4,13,Q154_1,7,True
3,ungs_2020,2020,Q154_2,10,Q154_1,9,True
4,ungs_2021,2021,Q154_2,13,Q154_3,7,True
5,ungs_2022,2022,Q154_1,15,Q154_3,15,True


In [5]:
## (Post-Processing) ##

# This code analyzes predicted labels per country-year from the predictions dataset,
# identifies the most and second most frequent labels for each country-year (Q154 and Q155, respectively),
# filters to keep only country-years present in the WVS dataset,
# then lists country-years missing a second most frequent label,
# and finally displays and saves the summarized results.

%run get_q154_q155_frequencies.py


Country-year pairs without a second most frequent label:
ETH - 2020
GTM - 2020
IRN - 2020
IRQ - 2018
JPN - 2019
KAZ - 2018
NGA - 2018
THA - 2018


,B_COUNTRY_ALPHA,A_YEAR,most_frequent_label,most_frequent_count,second_most_frequent_label,second_most_frequent_count,has_second_label
0,AND,2018,Q154_2,6,Q154_1,3,True
1,ARG,2017,Q154_2,2,Q154_1,1,True
2,ARM,2021,Q154_2,2,Q154_4,1,True
3,AUS,2018,Q154_1,9,Q154_2,3,True
4,BGD,2018,Q154_1,2,Q154_2,2,True
5,BOL,2017,Q154_1,4,Q154_4,3,True
6,BRA,2018,Q154_4,7,Q154_2,6,True
7,CAN,2020,Q154_1,4,Q154_2,1,True
8,CHL,2018,Q154_4,6,Q154_2,5,True
9,CHN,2018,Q154_1,14,Q154_2,3,True


In [6]:
## (Visualization -I) ##

# This code filters predicted Q154-related labels from the predictions dataset,
# calculates sentence counts and proportions of given response per country,
# prepares the data for ordered plotting,
# then creates and displays a scatter plot showing the proportion of Q154_1 sentences relative to total Q154 sentences by country,
# with point sizes representing sentence counts, x-axis showing the countries and y-axis the proportion of given response to total count responses.

%run visualize_preds_props.py

In [8]:
## (Visualization -II) ##

# This code loads World Values Survey (WVS) data and filters it by countries found in scored sentences.
# It processes Q154 survey responses to calculate per-country counts and proportions of respondents answering '1'.
# It prepares the country data in a categorical order for consistent plotting.
# Then it creates a scatter plot showing the proportion of Q154=1 responses per country, sized by total responses.
# The plot visually compares response distributions across countries with hover details and a clean layout.
# The x and y-axis labels are the same as the prediction visualization.

%run visualize_response_props.py

In [7]:
## (Visualization -III) ##

# This script compares proportions of a specific Q154 survey response between scored sentences and WVS data at the country-year level.
# It aggregates sentence counts and computes proportions per country-year in the scored predictions.
# It filters the WVS data to matching country-year pairs and computes weighted response proportions using survey weights.
# The code then merges both datasets and visualizes their proportions with connecting lines to compare distributions.
# The plot uses country-year labels on the x-axis and proportion values on the y-axis, with vertical lines visually highlighting differences between WVS survey and scored sentence proportions.
# Finally, it calculates and prints the Pearson correlation between the weighted WVS proportions and scored sentence proportions.

%run visualize_prop_diffs.py

Pearson correlation between WVS weighted and scored sentence proportions: 0.1734
